In [1]:
from matplotlib import pyplot as plt
# linear algebra
import numpy as np 

# data processing
import pandas as pd 

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import svm

#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Import Random Forest Model
from sklearn.ensemble import RandomForestClassifier

# csv file manipulation
import csv

In [2]:
# read training data 
df = pd.read_csv('./data/train.csv')

# read testing data
test_df = pd.read_csv('./data/test.csv')

In [3]:

# check for columns with missing values 
df.isnull().any()



tripid                       False
additional_fare               True
duration                      True
meter_waiting                 True
meter_waiting_fare            True
meter_waiting_till_pickup     True
pickup_time                  False
drop_time                    False
pick_lat                     False
pick_lon                     False
drop_lat                     False
drop_lon                     False
fare                          True
label                        False
dtype: bool

In [4]:
# replace missing values with mean of the particular feature

mean_additional_fare = df['additional_fare'].mean()
# print(mean_additional_fare)
mean_duration = df['duration'].mean()
mean_waiting = df['meter_waiting'].mean()
mean_waiting_fare = df['meter_waiting_fare'].mean()
mean_meter_till_pickup = df['meter_waiting_till_pickup'].mean()
mean_fare = df['fare'].mean()




df['additional_fare'] = np.where(df['additional_fare'].isnull(), mean_additional_fare, df['additional_fare'])
df['duration'] = np.where(df['duration'].isnull(), mean_duration, df['duration'])
df['meter_waiting'] = np.where(df['meter_waiting'].isnull(), mean_waiting, df['meter_waiting'])
df['meter_waiting_fare'] = np.where(df['meter_waiting_fare'].isnull(), mean_waiting_fare, df['meter_waiting_fare'])
df['meter_waiting_till_pickup'] = np.where(df['meter_waiting_till_pickup'].isnull(),
                                           mean_meter_till_pickup, df['meter_waiting_till_pickup'])
df['fare'] = np.where(df['fare'].isnull(), mean_fare, df['fare'])

In [5]:
# remove records with missing durations
# df = df.dropna(subset=['duration', 'fare'])
# df['duration'] = np.where(df['duration'].isnull(), mean_duration, df['duration'])
print(df['duration'].isnull().values.sum())

# x = type(df['duration']) == type(None)
# print(x)

0


In [6]:
df['trip_duration'] = df['duration']- (df['meter_waiting_till_pickup'] + df['meter_waiting'])

In [7]:
print(df['trip_duration'].isnull().values.sum())

0


In [8]:
test_df['trip_duration'] = test_df['duration']- (test_df['meter_waiting_till_pickup'] + test_df['meter_waiting'])
print(test_df['trip_duration'].isnull().values.sum())

0


In [9]:
df['trip_fare'] = df['fare'] -  (df['additional_fare'] + df['meter_waiting_fare'])
test_df['trip_fare'] = test_df['fare'] -  (test_df['additional_fare'] + test_df['meter_waiting_fare'])

print(df['trip_fare'].isnull().values.sum())
print(test_df['trip_fare'].isnull().values.sum())

0
0


In [10]:
mean_trip_duration = df['trip_duration'].mean()
mean_trip_fare = df['trip_fare'].mean()

df['trip_duration'] = np.where(df['trip_duration'].isnull(), mean_fare, df['trip_duration'])
df['trip_fare'] = np.where(df['trip_fare'].isnull(), mean_trip_fare, df['trip_fare'])

In [11]:
#split into data and label
Y = df['label'] # label set


In [12]:


# date formatting 
# df["start_date"] = pd.to_datetime(df["pickup_time"],format='%m/%d/%Y %H:%M')
# test_df["start_date"] = pd.to_datetime(test_df["pickup_time"],format='%m/%d/%Y %H:%M')


# # get the day of the week of the trip pickup date
# #Note that in Python Monday is 0 unlike Javascript style where Sunday is 0
# df['day_of_week'] = df['start_date'].dt.weekday
# test_df['day_of_week'] = test_df['start_date'].dt.weekday



tripIds = test_df['tripid'] # separate tripid to use when the results are written back to the submission

#remove attributes
test_df = test_df.drop(columns=['tripid', 'pickup_time', 'drop_time'])

X = df.drop(columns=['tripid', 'label', 'pickup_time', 'drop_time'])

X_test = test_df



In [13]:
#Feature Scaling
sc = StandardScaler()
X_train = sc.fit_transform(X)
X_test = sc.transform(X_test)

In [14]:
def write_output(Y_prediction):
    with open('sampleSubmission.csv', 'w', newline='') as csvfile:
        fieldnames = ['tripid', 'prediction']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        line = 0
    
        for w in Y_prediction:
            output = 1
            if (w == 'incorrect'):
                output = 0
            writer.writerow({'tripid': tripIds[line], 'prediction': output})
            line = line+1


In [15]:


def RFClassifier(X_train, y_train, X_test):
   
    #Create the Classifier
    clf=RandomForestClassifier(n_estimators=100)

    #Train the model using the training sets y_pred=clf.predict(X_test)
    clf.fit(X_train,y_train)

    y_pred=clf.predict(X_test)
    
    return y_pred
    

In [16]:
Y_pred = RFClassifier(X_train,Y, X_test)

write_output(Y_pred)

In [12]:
from sklearn.tree import DecisionTreeClassifier # Import Decision Tree Classifier

def decision_tree(X_train, y_train, X_test):
    # Create Decision Tree classifer object
    clf = DecisionTreeClassifier()

    # Train Decision Tree Classifer
    clf = clf.fit(X_train,y_train)

    #Predict the response for test dataset
    y_pred = clf.predict(X_test)
    
    return y_pred

tree_pred = decision_tree(X_train,Y, X_test)
write_output(tree_pred)

AttributeError: 'numpy.ndarray' object has no attribute 'columns'

In [36]:
X.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,trip_duration,trip_fare
0,10.500000,834.000000,56.000000,0.000000,64.000000,6.86252,79.8993,6.90330,79.8783,270.32,714.000000,259.820000
1,10.500000,791.000000,47.000000,0.000000,134.000000,6.88589,79.8984,6.91373,79.8923,197.85,610.000000,187.350000
2,10.500000,1087.000000,80.000000,0.000000,61.000000,6.90839,79.8651,6.93669,79.9146,301.64,946.000000,291.140000
3,10.500000,598.000000,271.000000,15.663800,68.000000,6.92570,79.8895,6.92748,79.8971,82.30,259.000000,56.136200
4,13.719651,1702.858077,629.074231,32.057666,112.466832,6.87441,79.8615,6.84478,79.9290,358.39,961.317014,312.612684


In [37]:
X_test.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,trip_duration,trip_fair,trip_fare
0,10.5,924,42,2.4486,148,6.83454,79.8750,6.77490,79.8840,289.27,734,276.3214,276.3214
1,10.5,4249,20,0.0000,91,6.91168,79.8723,6.55091,79.9706,1912.70,4138,1902.2000,1902.2000
2,10.5,1552,255,2.6588,23,6.92145,79.8478,6.90539,79.8989,394.00,1274,380.8412,380.8412
3,10.5,462,16,0.0000,198,6.77433,79.9416,6.80401,79.9407,154.32,248,143.8200,143.8200
4,10.5,814,392,12.3692,69,6.97968,79.9130,6.98875,79.8914,147.47,353,124.6008,124.6008


In [7]:
df['duration'].isnull().values.sum()
print(type(None))

<class 'NoneType'>
